#### Grey Wolf Optimization

The first step to define an optimizer is to define the domain on which the solution is searched. 

Second step is to define what we call error from optimum value or the loss function or the fitness function based on which solutions are judged. 

Third step is to generate random solutions from the domain and judge them based on loss function. 

Next, we update these solutions or solution based on an update expression which is expected to minimize the loss function further. 

In the present code, the loss function/error defined will be mostly based on the previous code for TDOA. We'll just have to make the form amenable to the algorithm we want to run. 

For the present solution we would generate some synthetic time of arrivals beforehand and then using that data we'll process it further. The identified domain is a rectangular domain of size 1km * 1km.


In [39]:
# importing required libraries 

import numpy as np
import matplotlib.pyplot as plt

In [59]:
# defining a suitable fitness function

def fitness_GWO(pos_matrix,ToA,wave_vel):
    '''The function returns the error value for a given position matrix(pos_matrix) consisting of source location as the first element
    and the receiver positions as all the rest elements. ToA is a vector of the time of arrival of seismic waves for receivers
    Wave_vel is the velocity of waves'''
    
    n_rec = np.shape(ToA)[0]
    TDoA = np.zeros(shape= (n_rec-1)) # created a new variable which will store time difference wrt the first receiver
    TDoA[:] = ToA[1:] - ToA[0]
    # Now getting the value for time of arrival from the position of source and receivers
    ToA_pos = np.zeros_like(ToA)
    pos_diff = np.zeros(shape = (np.shape(pos_matrix)[0],np.shape(pos_matrix)[1]-1))  # vector denoting the position of receivers wrt source
    pos_diff = pos_matrix[1:] - pos_matrix[0]
    ToA_pos[:] = (pos_diff[:,0]**2 + pos_diff[:,1]**2).reshape(2,1)/wave_vel
    TDoA_pos = np.zeros_like(TDoA)
    TDoA_pos[:] = ToA_pos[1:] - ToA_pos[0]
    error_vector = TDoA_pos - TDoA
    error = np.dot(error_vector.T,error_vector)
    
    return error

In [60]:
# Now generating the synthetic data for testing
wave_vel = 1 # m/s
n_rec = 2 # number of receivers
rec_pos_actual = np.random.normal(loc=0.0,scale =100,size = (n_rec,2))  # actual position for 2 receivers
src_pos_actual = np.array([100,100])
t_rec_ideal = np.zeros(shape =(n_rec,1))
# calculating the ideal time of arrival
t_rec_ideal[0:n_rec] = np.sqrt((rec_pos_actual[0:n_rec,1]-src_pos_actual[1])**2 + (rec_pos_actual[0:n_rec,0]-src_pos_actual[0])**2).reshape(n_rec,1)
#introducing noise to account for error in onset detection and sensor
noise_t = np.random.normal(loc=0.0,scale =0.2,size = (n_rec,1))
t_Rec_real = np.array(t_rec_ideal + noise_t)

In [61]:
# Now generating some random solutions for input and intializing number of wolves and their types, alpha, beta and delta are taken to be one each 

n_wolves = 5
initial_pos = -500 + 1000*(np.random.random(size=(n_wolves,n_rec+1,2)))  # in order to make sure that the spread is in entire domain


In [67]:
alpha_sol = np.zeros(shape=(n_rec+1,2))
alpha_fitness = 0.0
beta_sol = np.zeros(shape=(n_rec+1,2))
beta_fitness = 0.0
delta_sol = np.zeros(shape=(n_rec+1,2))
delta_fitness = 0.0
omega_sol = np.zeros(shape=(n_wolves-3,n_rec+1,2))

In [71]:
n_iter = 2 # number of iterations
wolves_pos = np.copy(initial_pos)# creating a single vector for positions as update expression will be similar
fitness_value = np.zeros(shape= n_wolves)

# First intialising the alpha, beta and delta
for j in range(n_wolves):
    fitness_value[j] = fitness_GWO(wolves_pos[j],t_Rec_real,wave_vel = wave_vel)
    
alpha_sol = initial_pos[np.where(fitness_value == np.min(fitness_value)),:,:]
alpha_fitness = fitness_value[np.where(fitness_value == np.min(fitness_value))]
fitness_value[np.where(fitness_value == np.min(fitness_value))] = fitness_value[np.where(fitness_value == np.min(fitness_value))]*30 # won't be minimum now hopefully

beta_sol = initial_pos[np.where(fitness_value == np.min(fitness_value)),:,:]
beta_fitness = fitness_value[np.where(fitness_value == np.min(fitness_value))]
fitness_value[np.where(fitness_value == np.min(fitness_value))] = fitness_value[np.where(fitness_value == np.min(fitness_value))]*30# won't be minimum now hopefully

delta_sol = initial_pos[np.where(fitness_value == np.min(fitness_value)),:,:]
delta_fitness = fitness_value[np.where(fitness_value == np.min(fitness_value))]
fitness_value[np.where(fitness_value == np.min(fitness_value))] = fitness_value[np.where(fitness_value == np.min(fitness_value))]*30 # won't be minimum now hopefully

array([1.52206142e+10])